Gold nanoparticle builder
====

## Libraries

In [37]:
import os,sys
import numpy as np

from ase.io import write
from ase.cluster import wulff_construction 
from ase.parallel import paropen
from ase.utils import basestring

from itertools import cycle, product, permutations
from scipy.optimize import minimize_scalar

## Using python library ASE <a name="S1"></a>

The [**Atomic Simulation Environment** (ASE)](https://wiki.fysik.dtu.dk/ase/index.html) is a set of tools and modules for setting up, manipulating, running, visualizing. and analyzing atomistic simulations. The code is freely available under the [GNU LGPL license](https://wiki.fysik.dtu.dk/ase/development/licenseinfo.html#license-info).

To create our gold nanoparticle (AuNP) we use the `ase.cluster` package. We employ the `wulff_construction` method modifying it adequately for our purpose.

### Wulff construction <a name="S1.1"></a>

The [**Wulff construction**](https://en.wikipedia.org/wiki/Wulff_construction) is a recursive method to determine the equilibrium shape of a crystal with a known mass by minimizing the surface energy.

To set up a Wulff construction an important information is **the ratio between surface energies** of the crystalline planes chosen as reference. 

> **Note**: `esurf` variable is the ratio between the surface tension of the two crystalline planes, namely $\mathrm{\gamma_{SV}^{\{111\}}}/\mathrm{\gamma_{SV}^{\{100\}}}=0.96$ [calculated by Heinz et al.](https://pubs.acs.org/doi/abs/10.1021/jp801931d).
> If you want to vary the AuNP surface shape, you could modify `esurf` 

In [38]:
surfaces = [(1, 0, 0), (1, 1, 1)]            # Surface families (Miller's indices) {1,0,0} {1,1,1}
esurf = [1.0, 0.9]                           # Surface energies (guess)

lc = 4.07825                                 # [Å] lattice constant
size = 4000                                  # Number of atoms (guess)

# building gold NP
au_np = wulff_construction('Au', surfaces, esurf, size, 'fcc', rounding='above', 
                           latticeconstant=lc, debug=True) # `debug=True` makes the function "noisy" 

Wulff: Aiming for cluster with 4000 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 4033 atoms


### Build NP with desired diameter <a name="S1.2"></a>

We have to modify the Wulff construction because it is build from whole layers, so it is not possible to impose a desired diameter. 

To set up the diameter, we write a function that minimizes the absolute value of the difference between the desired diameter and the equivalent diameter of a sphere with the same mass as the cluster (`au_np`).  

In [39]:
D = 40                                   # Desired diameter [Å] 

surfaces = [(1, 0, 0), (1, 1, 1)]        # Surface families (Miller's indices) {1,0,0} {1,1,1}
esurf = [1.0, 0.9]                       # Surface energies (guess). 

lc = 4.07825                             # [Å] lattice constant
size = 4000                              # Number of atoms (guess)

def np_make(size, lc = lc, surfaces = surfaces, esurf = esurf, debug=False): 
    return wulff_construction('Au', surfaces, esurf, size, 'fcc', rounding='above', latticeconstant=lc, debug=True)
    
def diameter(size):
    np_ = np_make(size)           
    return np_.get_diameter()

def f(size):
    return abs(diameter(size)-D)

opt = minimize_scalar(f, bounds=(5, 5e4), method='bounded', options=dict(disp=2, maxiter=10))

print(opt)

Wulff: Aiming for cluster with 19101 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 20069 atoms
Wulff: Aiming for cluster with 30903 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 33469 atoms
Wulff: Aiming for cluster with 11807 atoms (above)
Making a larger cluster.
Choosing larger cluster with 13233 atoms
Wulff: Aiming for cluster with 7299 atoms (above)
Making a larger cluster.
Choosing larger cluster with 8715 atoms
Wulff: Aiming for cluster with 4513 atoms (above)
Making a larger cluster.
Choosing larger cluster with 4957 atoms
Wulff: Aiming for cluster with 2791 atoms (above)
Making a larger cluster.
Choosing larger cluster with 2951 atoms
Wulff: Aiming for cluster with 1726 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 1925 atoms
Wulff: Aiming for cluster with 1069 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 1139 atoms
Wulff: Aiming for cluster with 1963 atoms (above)
Making a larger clus

In [40]:
au_np_d40 = np_make(opt.x, debug=True)

Wulff: Aiming for cluster with 1726 atoms (above)
Making a smaller cluster.
Choosing larger cluster with 1925 atoms


### Surface planes <a name="S1.3"></a> 

To identify in which plane the surface atoms belong to, we use the geometric definition of a plane in the space. Namely, a point belongs to a plane $\Sigma$  the following equation is satisfied:

$P\in \Sigma \quad \Rightarrow \quad \left\| \left( \mathbf{r}-\mathbf{r}_0 \right)\cdot \mathbf{n} \right\| = 0 \quad\text{where}\quad \mathbf{r} = O-P,\; \mathbf{r}_0 \in \Sigma $

<p align="left">
<img src="../img/def_plane.png" width="30%" height ="30%" style="background-color:white">
</p>

The `cluster` object has the property `cluster.surfaces` which give the index miller of all faces:
```
>>> au_np.surfaces[:5]    # print the first 5 planes
>>> array([[ 1,  0,  0],
           [ 1,  1,  1],
           [ 0,  0, -1],
           [ 0, -1,  0],
           [-1,  0,  0]])

```

And we can get all the plane direction with the method(function) `cluster.miller_to_direction()`:
```
>>> au_np.miller_to_direction(s[:5])                  # print the direction of first 5 planes
>>> array([[ 0.37796447,  0.        ,  0.        ],
           [ 0.37796447,  0.37796447,  0.37796447],
           [ 0.        ,  0.        , -0.37796447],
           [ 0.        , -0.37796447,  0.        ],
           [-0.37796447,  0.        ,  0.        ]])
```

Thus our function is straight forward:

0. We translate the NP to obtain the NP center in $(0, 0, 0)$
1. For each surface in `cluster.surfaces` we compute the direction $\mathbf{n}$ with `cluster.miller_to_direction()`
2. Since we have different layers in each direction, we get the distance of the last projecting all the atoms along the $\mathbf{n}$ and calculate the maximum, namely $d=\max(\mathbf{r}_i\cdot\mathbf{n})$
3. We build $\mathbf{r}_0 = d\mathbf{n}$ which for definition belongs to the surface
4. Finally, we find the surface atoms with the logic operation $\left\| \left( \mathbf{r}_i-\mathbf{r}_0 \right)\cdot \mathbf{n} \right\| < 1e-10$



The result will be stored in the dictionary `cluster.arrays["aname"]` as a list of atom names. 
In this specific case, we use:
* `AUb` gold atoms in the bulk
* `AU1` gold atoms belonging to the surfaces {1 0 0}
* `AU2` gold atoms belonging to the surfaces {1 1 1}
* `AU3` gold atoms belonging to both the surfaces {1 0 0} and {1 1 1}

In [41]:
_debug = False

def surf_identifier(cluster, tol=1e-10, debug=False):
    """Create a cluster using the Wulff construction.

    A cluster is created with approximately the number of atoms
    specified, following the Wulff construction, i.e. minimizing the
    surface energy of the cluster.

    Parameters:

    size: The desired number of atoms.

    cluster: Cluster ASE object.
    
    tol (optional): Tolerance.

    debug (optional): If non-zero, information about the identification.
    """
    global _debug
    _debug = debug
    
    # add "aname" key with the atom name + "b"
    cluster.arrays["aname"] = np.r_[[symbol.upper()+"b" for symbol in cluster.get_chemical_symbols()]].astype(dtype=str)
    
    # Find all possible plane that belong to the families {1 0 0}, {1 1 1}
    p100 = np.r_[[list(mask) for mask in permutations([1,0,0])]]
    p111 = np.r_[[list(mask) for mask in product(*[[1,-1]]*3)]]
    
    if _debug:
        ntot = len(cluster.get_chemical_symbols())
        n_a = {"1": 0, "2": 0, "3": 0}
    
    for s in cluster.surfaces:
        flag = None
        for m1, m2 in zip(cycle(p100), p111):
            if (abs(s[0]) == m1[0]) & (abs(s[1]) == m1[1]) & (abs(s[2]) == m1[2]):  
                flag = "1"
                break
            elif (abs(s[0]) == m2[0]) & (abs(s[1]) == m2[1]) & (abs(s[2]) == m2[2]): 
                flag = "2"
                break
                
        n = cluster.miller_to_direction(s)
        c = cluster.get_positions().mean(axis=0)
        r = cluster.get_positions() - c
        d = np.dot(r, n).max()
        r0 = n * d
        
        mask = (abs(np.dot(r - r0, n)) < tol)
        lastchar = np.r_[[i[-1] for i in cluster.arrays["aname"]]]
        if _debug:
            n_a["3"] += sum(mask & (lastchar != "b")) 
        cluster.arrays["aname"][mask & (lastchar != "b")] = [i[:-1]+"3" for i in cluster.arrays["aname"][mask & (lastchar != "b")]]   
        if _debug:
            n_a[flag] += sum(mask & (lastchar == "b")) 
        cluster.arrays["aname"][mask & (lastchar == "b")] = [i[:-1]+flag for i in cluster.arrays["aname"][mask & (lastchar == "b")]]   
    if _debug:
        print("Cluster of %d atoms" % ntot)
        print("\t%d atoms belong to {1 0 0} surface" % n_a["1"])
        print("\t%d atoms belong to {1 1 1} surface" % n_a["2"])
        print("\t%d atoms belong to both surfaces" % n_a["3"])
        print("\t%d atoms remaining" % (ntot-n_a["1"]-n_a["2"]-n_a["3"]))
    return cluster

Let's run it

In [42]:
au_np_surf_d40 = surf_identifier(au_np_d40, debug=True)

Cluster of 1925 atoms
	204 atoms belong to {1 0 0} surface
	432 atoms belong to {1 1 1} surface
	180 atoms belong to both surfaces
	1109 atoms remaining


### Save as PDF <a name="S1.4"></a> 

Now we are quite far from the ASE packages philosophy, and its `write` function will arise an error.
To solve this drawback, we will write a new "write" function `write_proteindatabank()` able to save the NP in .PDB file and use `au_np_surf.arrays["aname"]` as atom names instead of the chemical symbol.

In [43]:
def write_proteindatabank(fileobj, images, write_arrays=True, change_name=False, author=None, header=None, 
                          title=None):
    """Write images to PDB-file."""
    if isinstance(fileobj, basestring):
        fileobj = paropen(fileobj, 'w')

    if hasattr(images, 'get_positions'):
        images = [images]
        
    if isinstance(header, basestring) or isinstance(header, list):
        format="HEADER     %s\n"
        if isinstance(header, basestring):
            header = [header]
        for i, h in zip(range(len(header)), header):
            fileobj.write(format % (h))
                        
    if isinstance(title, basestring) or isinstance(title, list):
        format="TITLE %4d %s\n"
        if isinstance(title, basestring):
            title = [title]
        for i, t in zip(range(len(title)), title):
            fileobj.write(format % (i+1, t))
    
    if isinstance(author, basestring) or isinstance(author, list):
        format="AUTHOR%4d %s\n"
        if isinstance(author, basestring):
            author = [author]
        for i, a in zip(range(len(author)), author):
            fileobj.write(format % (i+1, a))
            
    # Crystal info:
    symbols = ""
    for symb in np.unique(images[0].get_chemical_symbols()):
        symbols += symb
        
    brav_latt = images[0].lattice_basis
    unitcell = images[0].lattice_basis.diagonal() * images[0].atomic_basis
    remark = """%10s Nano Particle
Bravais lattice = [[%8.3f, %8.3f, %8.3f]
                   [%8.3f, %8.3f, %8.3f]
                   [%8.3f, %8.3f, %8.3f]]
Atomic Basis:
""" % (symbols, brav_latt[0,0], brav_latt[0,1], brav_latt[0,2], brav_latt[1,0],
          brav_latt[1,1], brav_latt[1,2], brav_latt[2,0], brav_latt[2,1], brav_latt[2,2])
    for i, atom in zip(range(len(unitcell)), unitcell):
        remark += "         atom%d = [%8.3f, %8.3f, %8.3f]\n" % (i, atom[0], atom[1], atom[2])
    
    format="REMARK%4d %s\n"
    remark = remark.split("\n")
    for i, r in zip(range(len(remark)), remark):
        fileobj.write(format % (i+1, r))
    
    rotation = None
    if images[0].get_pbc().any():
        from ase.geometry import cell_to_cellpar, cellpar_to_cell

        currentcell = images[0].get_cell()
        cellpar = cell_to_cellpar(currentcell)
        exportedcell = cellpar_to_cell(cellpar)
        rotation = np.linalg.solve(currentcell, exportedcell)
        # ignoring Z-value, using P1 since we have all atoms defined explicitly
        format = 'CRYST1%9.3f%9.3f%9.3f%7.2f%7.2f%7.2f P 1\n'
        fileobj.write(format % (cellpar[0], cellpar[1], cellpar[2],
                                cellpar[3], cellpar[4], cellpar[5]))

    #format = ('ATOM  %5d %4s MOL     1    %8.3f%8.3f%8.3f%6.2f%6.2f'
    #          '          %2s  \n')

    format = ('%6s%5d %4s %4s%1s%4d    %8.3f%8.3f%8.3f%6.2f%6.2f          %2s  \n')
    
    # RasMol complains if the atom index exceeds 100000. There might
    # be a limit of 5 digit numbers in this field.
    MAXNUM = 100000

    symbols = np.r_[images[0].get_chemical_symbols()]
    aname = images[0].arrays['aname']
    natoms = len(symbols)

    for n, atoms in enumerate(images):
        fileobj.write('MODEL     ' + str(n + 1) + '\n')
        p = atoms.get_positions()
        occupancy = np.ones(len(atoms))
        bfactor = np.zeros(len(atoms))

        if write_arrays:
            if 'occupancy' in atoms.arrays:
                occupancy = atoms.get_array('occupancy')
            if 'bfactor' in atoms.arrays:
                bfactor = atoms.get_array('bfactor')
        if rotation is not None:
            p = p.dot(rotation)
        nb, n1, n2, n3 = [0, 0, 0, 0]
        for a in range(natoms):
            x, y, z = p[a]
            occ = occupancy[a]
            bf = bfactor[a]
            if change_name: # <-------------- this option was deeded only to solve a bug of the NGL viewer 
                if aname[a][-1] == "b":
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, aname[a], "NP", " ", 1,
                                            x, y, z, occ, bf, symbols[a].upper()))
                elif aname[a][-1] == "1":
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, "O", "NP", " ", 2,
                                            x, y, z, occ, bf, "O"))
                elif aname[a][-1] == "2":
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, "C", "NP", " ", 3,
                                            x, y, z, occ, bf, "C"))
                elif aname[a][-1] == "3":
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, "He", "NP", " ", 4, 
                                            x, y, z, occ, bf, "He"))
                    
            else:
                if aname[a][-1] == "b":
                    nb+=1
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, aname[a], "AUC ", " ", (nb) % MAXNUM,
                                            x, y, z, occ, bf, symbols[a].upper()))
                elif aname[a][-1] == "1":
                    n1+=1
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, aname[a], "AU10", " ", (n1) % MAXNUM,
                                            x, y, z, occ, bf, symbols[a].upper()))
                elif aname[a][-1] == "2":
                    n2+=1
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, aname[a], "AU11", " ", (n2) % MAXNUM,
                                            x, y, z, occ, bf, symbols[a].upper()))
                elif aname[a][-1] == "3":
                    n3+=1
                    fileobj.write(format % ("ATOM  ",(a+1) % MAXNUM, aname[a], "AUS ", " ", (n3) % MAXNUM,
                                            x, y, z, occ, bf, symbols[a].upper()))
                    
        fileobj.write('ENDMDL\n')
    fileobj.close()

In [44]:
# save
savefolder = "../simulations/0-Packmol/inputs_self-assembly"
namefile = os.path.join(savefolder, "AuNP_ASE_wulff_surf_d%d.pdb" % au_np_surf_d40.get_diameter() )
author=["Roberta Cappabianca", "Paolo De Angelis", "Annalisa Cardellini"]
write_proteindatabank(namefile, au_np_surf_d40, header="NP made with modified ASE python module", author=author)

print("-"*80)
print("\033[1;32m"+"file saved: " + "\033[1;34m.%s%s\033[0;0m" % (os.sep, namefile))

--------------------------------------------------------------------------------
file saved: ./../simulations/0-Packmol/inputs_self-assembly/AuNP_ASE_wulff_surf_d39.pdb
